In [155]:
import json
import read_data
from collections import Counter

In [159]:
data = read_data.load_trn_data()

In [160]:
def get_input_data(data):
    result = []
    for sent in data:
        sent_list = []
        
        tok_idx = []
        tok_str = []
        tok_arg = []
        for token in sent:
            tok_idx.append(token[0])
            tok_str.append(token[1])
            tok_arg.append(token[2])
            
        sent_list.append(tok_idx)
        sent_list.append(tok_str)
        sent_list.append(tok_arg)
        result.append(sent_list)
    return result
        
input_data = get_input_data(data)

In [164]:
arg_pos = []
for s_idx in range(len(input_data)):
    sent = input_data[s_idx]
    anno = sent[2]
    args = []
    for i in range(len(anno)):
        a = anno[i]
        if a !='-':
            args.append(i)
            
    conll = read_data.get_nlp_for_trn(s_idx)

    for i in args:
        pos = conll[i][-1]
        arg_pos.append(pos)

print(len(list(set(arg_pos))))
print(len(arg_pos))
print(Counter(arg_pos))     

26
124873
Counter({'NP_SBJ': 41952, 'NP_OBJ': 37190, 'NP_AJT': 36104, 'NP_CMP': 2996, 'NP': 1188, 'VNP': 801, 'NP_MOD': 736, 'VP_AJT': 699, 'VP_OBJ': 639, 'VP': 634, 'NP_CNJ': 463, 'VP_SBJ': 368, 'VNP_OBJ': 276, 'AP': 182, 'VNP_MOD': 162, 'VP_MOD': 114, 'VNP_AJT': 104, 'VNP_CMP': 85, 'VNP_SBJ': 80, 'VP_CMP': 46, 'DP': 19, 'AP_AJT': 17, 'NP_INT': 12, 'IP': 4, 'AP_OBJ': 1, 'X': 1})


In [ ]:
for s_idx in range(len(data)):
    sent = data[s_idx]
    conll = read_data.get_nlp_for_trn(s_idx)
    
    for t_idx in range(len(data)):
        

In [119]:
def get_head_tail(t_idx, conll):
    tail = False
    head_of_head_arc = False
    head_arc = int(conll[t_idx][8])
    head_of_head_arc = int(conll[head_arc][8])
    head = conll[head_arc][-1]
    
    for i in range(len(conll)):
        token = conll[i]
        if int(token[8]) == t_idx:
            child = token[-1]
            child_arc = int(token[8])
            if child.startswith('V'):
                tail = child          
                    
    return head, tail

In [123]:
pos_list = []
neg_pos = []
h,t,neg = 0,0,0
for s_idx in range(len(data)):
    sent = data[s_idx]
    arg_tokens = []
    
    tokens = []
    for t_idx in range(len(sent)):

        
        tok = sent[t_idx]
        tokens.append(tok[1])
        if tok[2] != '-':
            arg_tokens.append(t_idx)
    text = ' '.join(tokens)
    conll = read_data.get_nlp_for_trn(s_idx)
    
    add = False
    for t_idx in arg_tokens:
        head, tail = get_head_tail(t_idx, conll)
#         print(tail)
        if head.startswith('V'):
            pos = head
            pos_list.append(pos)
            add = True
            h +=1
        else:
            if tail:
                if tail.startswith('V'):
                    pos = tail
                    pos_list.append(pos)
                    add = True
                    t +=1
        if add == False:
            neg_pos.append(head)
            neg +=1

print(len(list(set(pos_list))))
print(len(pos_list))
print(Counter(pos_list))
        
print(len(list(set(neg_pos))))
print(len(neg_pos))
print(Counter(neg_pos))

print(h)
print(t)
print(neg)

13
123037
Counter({'VP': 79093, 'VP_MOD': 36105, 'VP_OBJ': 1830, 'VP_CMP': 1744, 'VNP': 1653, 'VP_AJT': 1076, 'VP_SBJ': 576, 'VNP_CMP': 395, 'VNP_MOD': 321, 'VNP_OBJ': 145, 'VNP_AJT': 66, 'VNP_SBJ': 25, 'VP_CNJ': 8})
11
630
Counter({'NP_AJT': 228, 'NP_SBJ': 125, 'NP': 117, 'NP_OBJ': 101, 'NP_MOD': 20, 'X': 14, 'NP_CNJ': 11, 'NP_CMP': 6, 'AP': 6, 'NP_PRN': 1, 'NP_INT': 1})
121708
1329
630


In [167]:
def get_arg(t_idx, conll):
    args = []
    for i in range(len(conll)):
        tok = conll[i]
        if int(tok[8]) == t_idx:
            arg_pos = tok[-1]
            if arg_pos[:2] == 'NP':
                args.append(i)
    return args

In [166]:
a = 'NP_SBJ'
print(a[:2])

NP


In [180]:
posi = list(set(pos_list))
scores = 0
n = 0
for s_idx in range(len(data)):
    sent = data[s_idx]
    conll = read_data.get_nlp_for_trn(s_idx)
    
    verbs = []
    for i in range(len(conll)):
        tok = conll[i]
        if tok[-1] in posi:
            verbs.append(i)
            
    pred_args = []
    for i in verbs:
        args = get_arg(i, conll)
        pred_args += args
        
    anno_args = []
    for i in range(len(sent)):
        tok = sent[i]
#         print(tok)
        arg = tok[2]
#         print(arg)
        if arg != '-':
            anno_args.append(i)

    
    d = set(pred_args).intersection(anno_args)
    e = set(anno_args).intersection(pred_args)
    if len(d) == 0:
        score = 0
    else:
        score = len(d) / len(anno_args)

    scores += score
    n += 1
print(scores / n)

0.946590308866747
